In [1]:
import numpy as np
import random
import pandas as pd

import sys
sys.path.append('C:/Users/ug033207/dynamic scheduling/dynamic-scheduling/functions')
import vizings_schedule
from yeni_model import yeni_model
from yeni_model_nlp import yeni_model_nlp



In [2]:
n = 8
teams = list(range(1,n+1))
days = list(range(1,n))
#days = list(range(n-1,0,-1))
#weights_list = [round(1/i) for i in range(1, n)]
#print(weights_list)
number_of_matches = int((n*(n-1))/2)
result_df, result_matrix = vizings_schedule.generate_results(teams)
print(result_matrix)
simulation = []

for i in range(1,1001):
    timetable = vizings_schedule.vizing(n)
    each_sim = vizings_schedule.calculate_obj_value(n, timetable, result_matrix)
    simulation.append(each_sim)

canon = vizings_schedule.canonical(n)
obj_canon = vizings_schedule.calculate_obj_value(n, timetable, result_matrix)
print(canon)

[[0 1 3 0 3 1 3 3]
 [1 0 0 1 3 3 3 3]
 [0 3 0 1 3 3 3 0]
 [3 1 1 0 1 1 0 3]
 [0 0 0 1 0 1 1 0]
 [1 0 0 1 1 0 1 0]
 [0 0 0 3 1 1 0 1]
 [0 0 3 0 3 3 1 0]]
[[0 0 0 0 0 0 0 0 0]
 [0 0 1 2 7 3 6 4 5]
 [0 1 0 5 4 2 7 6 3]
 [0 2 5 0 1 6 4 3 7]
 [0 7 4 1 0 5 3 2 6]
 [0 3 2 6 5 0 1 7 4]
 [0 6 7 4 3 1 0 5 2]
 [0 4 6 3 2 7 5 0 1]
 [0 5 3 7 6 4 2 1 0]]


In [3]:
# Burada result matrixi oluşturuyorum.
result_array = np.zeros((n, n), dtype=int)
for _, row in result_df.iterrows():
    result = row['Result']
    team1 = row['Team 1']
    team2 = row['Team 2']

    index_team1 = teams.index(team1)
    index_team2 = teams.index(team2)

    if team1 == result:
        result_array[index_team1, index_team2] = 3
    elif team2 == result:
        result_array[index_team2, index_team1] = 3
    else:
        result_array[index_team1, index_team2] = 1
        result_array[index_team2, index_team1] = 1
print(result_array)
data = pd.DataFrame(result_array, columns=teams, index=teams)


output, d1_df, d2_df, y_df, z_df, obj_value, l, l2= yeni_model(teams, days, data)
ratings1 = {i: 0 for i in teams}
sonuc1 = np.array(output)



for d in days:
    obj_count =0
    for row in sonuc1:
        day,team1,team2 = row
        if d==day:
            obj_count +=(abs(ratings1[team1] - ratings1[team2]))
            ratings1[team1] += result_matrix[team1-1,team2-1]
            ratings1[team2] += result_matrix[team2-1,team1-1]

[[0 1 3 0 3 1 3 3]
 [1 0 0 1 3 3 3 3]
 [0 3 0 1 3 3 3 0]
 [3 1 1 0 1 1 0 3]
 [0 0 0 1 0 1 1 0]
 [1 0 0 1 1 0 1 0]
 [0 0 0 3 1 1 0 1]
 [0 0 3 0 3 3 1 0]]
Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-15
Read LP format model from file C:\Users\ug033207\AppData\Local\Temp\tmpjhx6ekp_.pyomo.lp
Reading time = 0.02 seconds
x1: 1456 rows, 980 columns, 8848 nonzeros
Set parameter TimeLimit to value 14400
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1456 rows, 980 columns and 8848 nonzeros
Model fingerprint: 0x1d322f67
Variable types: 392 continuous, 588 integer (196 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+0

In [4]:
output2, d1_df, d2_df, y_df, z_df, obj_value2, l, l2= yeni_model_nlp(teams, days, data)
sonuc = np.array(output2)
ratings = {i: 0 for i in teams}

objective_values = []


for d in days:
    obj_count =0
    for row in sonuc:
        day,team1,team2 = row
        if d==day:
            obj_count +=(abs(ratings[team1] - ratings[team2]))
            ratings[team1] += result_matrix[team1-1,team2-1]
            ratings[team2] += result_matrix[team2-1,team1-1]
    print(ratings)
    objective_values.append(obj_count)
a = sum(objective_values)

0.14285714285714285
0.16666666666666666
0.2
0.25
0.3333333333333333
0.5
1.0
Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-15
Read LP format model from file C:\Users\ug033207\AppData\Local\Temp\tmp0yy4_s58.pyomo.lp
Reading time = 0.02 seconds
x1: 1456 rows, 980 columns, 8848 nonzeros
Set parameter TimeLimit to value 14400
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1456 rows, 980 columns and 8848 nonzeros
Model fingerprint: 0x6ddd6e65
Variable types: 392 continuous, 588 integer (196 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 868 rows and 112 columns
Presolve time: 0.02s
Pres

In [5]:
print(simulation)
simulation = np.array(simulation)
mean = np.mean(simulation)
print(mean)
print(obj_canon)
print(a)
print(obj_value)

[67, 75, 86, 68, 67, 77, 76, 75, 62, 87, 69, 66, 69, 60, 65, 68, 73, 67, 83, 69, 64, 69, 84, 72, 78, 71, 56, 71, 76, 65, 59, 80, 84, 82, 70, 77, 74, 69, 71, 71, 79, 81, 65, 80, 80, 68, 84, 63, 70, 60, 77, 88, 57, 85, 72, 64, 75, 74, 78, 75, 70, 70, 73, 68, 67, 70, 68, 75, 64, 88, 60, 71, 67, 79, 69, 78, 74, 71, 71, 82, 80, 89, 65, 77, 78, 75, 72, 71, 68, 81, 55, 69, 72, 73, 73, 69, 85, 82, 76, 67, 68, 71, 85, 70, 69, 72, 82, 85, 67, 60, 83, 76, 73, 78, 60, 65, 68, 75, 78, 67, 72, 63, 77, 75, 71, 86, 67, 77, 83, 63, 66, 67, 71, 70, 64, 67, 65, 76, 73, 74, 81, 84, 74, 60, 70, 75, 65, 75, 68, 76, 64, 71, 73, 64, 64, 82, 72, 86, 72, 76, 77, 59, 66, 75, 68, 66, 76, 75, 75, 73, 70, 80, 69, 84, 73, 70, 79, 68, 58, 79, 73, 83, 68, 86, 89, 79, 62, 75, 62, 67, 78, 72, 83, 64, 68, 78, 68, 86, 64, 82, 78, 59, 65, 77, 70, 63, 71, 69, 74, 67, 73, 86, 71, 80, 72, 84, 83, 76, 70, 67, 65, 70, 70, 72, 66, 60, 69, 73, 74, 67, 66, 65, 61, 70, 83, 77, 63, 63, 62, 73, 83, 77, 70, 77, 60, 68, 70, 68, 71, 82,

teams = list(range(1,n+1))
days = list(range(1,n))
non_zero_entries = timetable[1:, 1:]

new_format_list = []

# Iterate through the timetable matrix to extract day, team1, and team2 information
for team1 in range(n):
    for team2 in range(n):
        day = non_zero_entries[team2, team1]
        new_format_list.append([day, team1, team2])

# Convert the list to a NumPy array
new_format_array = np.array(new_format_list)
new_format_array[:, 1:] += 1
new_format_array = new_format_array[new_format_array[:, 0].argsort()]
new_format_array = new_format_array[new_format_array[:, 2] < new_format_array[:, 1]]
solution_df = pd.DataFrame(new_format_array, columns=['Day', 'Team 1', 'Team 2'])
relevant_matches = pd.DataFrame(columns=['Team 1', 'Team 2', 'Day'])
for _, row in solution_df.iterrows():
    day = row['Day']
    team1 = row['Team 1']
    team2 = row['Team 2']
    if team1 < team2:
        relevant_matches = pd.concat([relevant_matches, pd.DataFrame({'Team 1': [team1], 'Team 2': [team2], 'Day': [day]})],
                                                    ignore_index=True)
    else:
        relevant_matches = pd.concat([relevant_matches, pd.DataFrame({'Team 1': [team2], 'Team 2': [team1], 'Day': [day]})],
                                                    ignore_index=True)
result_array = np.zeros((n, n), dtype=int)
for _, row in result_df.iterrows():
    result = row['Result']
    team1 = row['Team 1']
    team2 = row['Team 2']

    index_team1 = teams.index(team1)
    index_team2 = teams.index(team2)

    if team1 == result:
        result_array[index_team1, index_team2] = 3
    elif team2 == result:
        result_array[index_team2, index_team1] = 3
    else:
        result_array[index_team1, index_team2] = 1
        result_array[index_team2, index_team1] = 1
print(result_array)
data = pd.DataFrame(result_array, columns=teams, index=teams)



output, d1_df, d2_df, y_df, z_df, obj_value, l, l2= yeni_model(teams,days,data)

import matplotlib.pyplot as plt
import numpy as np

data_model = {4: 6, 6: 8, 8: 27, 10: 35, 12: 64, 14: 102, 20: 537}
data_sim = {4: 7.8, 6: 23.2, 8: 62.5, 10: 106.2, 12: 183, 14: 283.3, 20: 749.8}

# Extract keys and values
keys = list(data_model.keys())
values_model = list(data_model.values())
values_sim = list(data_sim.values())

# Create a bar chart
bar_width = 0.35
index = np.arange(len(keys))

fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figure size

bar1 = ax.bar(index, values_model, bar_width, label='Model', color='skyblue', edgecolor='black')
bar2 = ax.bar(index + bar_width, values_sim, bar_width, label='Mean of 1000 schedules', color='lightcoral', edgecolor='black')

# Set labels and title
ax.set_xlabel('Number of Teams', fontsize=14)
ax.set_ylabel('Objective Values', fontsize=14)
ax.set_title('Comparison Model with 1000 Random Schedules', fontsize=16)
ax.set_xticks(index + bar_width/2)
ax.set_xticklabels(keys)
ax.legend()

# Add grid lines
ax.grid(axis='y', linestyle='--', alpha=0.7)

# Add value annotations on top of the bars
for i, v in enumerate(values_model):
    ax.text(i, v + 10, str(v), ha='center', va='bottom', color='black', fontweight='bold', fontsize=10)

for i, v in enumerate(values_sim):
    ax.text(i + bar_width, v + 10, str(v), ha='center', va='bottom', color='black', fontweight='bold', fontsize=10)

# Show the plot
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()


obj_data = (7730, 7418, 630, 576, 558, 554, 536 ,528, 518, 508, 506, 504, 497, 484, 465, 460, 449, 442, 423, 414, 412, 408, 407, 406, 401, 400, 398, 397, 395, 393, 391, 389, 384, 382, 375, 369, 367, 361)
time_change = (12.1, 12.5, 15, 16, 87, 89, 148, 830, 861, 892, 893, 894, 932, 975, 1005, 2017, 2887, 2928, 2929, 2977, 3017, 3018, 3168, 3220, 3382, 3647, 3648, 3952, 4252, 6514, 6515, 6549, 6563, 6564, 7025, 7080,9485,9869)
print(len(obj_data))
print(len(time_change))
obj_data_dict = {i: obj_data[i] for i in range(len(obj_data))}
time_change_dict = {i: time_change[i] for i in range(len(time_change))}

# Create a numpy array with two columns
data_array = np.column_stack((time_change, obj_data))

print(data_array)



import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D

# Your data
obj_data = (7730, 7418, 630, 576, 558, 554, 536, 528, 518, 508, 506, 504, 497, 484, 465, 460, 449, 442, 423, 414, 412, 408, 407, 406, 401, 400, 398, 397, 395, 393, 391, 389, 384, 382, 375, 369, 367, 361, 361)
time_change = (12.1, 12.5, 15, 16, 87, 89, 148, 830, 861, 892, 893, 894, 932, 975, 1005, 2017, 2887, 2928, 2929, 2977, 3017, 3018, 3168, 3220, 3382, 3647, 3648, 3952, 4252, 6514, 6515, 6549, 6563, 6564, 7025, 7080, 9485, 9869, 14400)

# Create a numpy array with two columns
data_array = np.column_stack((time_change, obj_data))

# Set Seaborn style
sns.set(style="whitegrid")

# Create a figure and axis object
fig, ax = plt.subplots(figsize=(10, 6))

# Plot the data using Seaborn with specified colors
line = sns.lineplot(x=time_change, y=obj_data, color='skyblue', ax=ax)
scatter = sns.scatterplot(x=time_change, y=obj_data, color='red', ax=ax, s=40, zorder=10)

# Set the y-axis limits
ax.set_ylim(250, 800)

# Set plot labels and title
ax.set_xlabel('Time (seconds)')
ax.set_ylabel('Objective Value')
ax.set_title('Change of Objective Values throughout the iterations (18 teams)')

# Add legend with a note
legend_labels = {'first': 'First iteration: 7730', 'second': 'Second iteration: 7418'}

# Create custom legend handler for red dots
legend_elements = [Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label=label) for label in legend_labels.values()]

# Add legend with custom elements
ax.legend(handles=[line, *legend_elements])

# Show plot
plt.show()


non_zero_entries = timetable[1:, 1:]

new_format_list = []

# Iterate through the timetable matrix to extract day, team1, and team2 information
for team1 in range(n):
    for team2 in range(n):
        day = non_zero_entries[team2, team1]
        new_format_list.append([day, team1, team2])

# Convert the list to a NumPy array
new_format_array = np.array(new_format_list)
new_format_array[:, 1:] += 1
new_format_array = new_format_array[new_format_array[:, 0].argsort()]
new_format_array = new_format_array[new_format_array[:, 2] < new_format_array[:, 1]]
solution_df = df = pd.DataFrame(new_format_array, columns=['Day', 'Team 1', 'Team 2'])

path = ('C:/Users/ug033207/vizings deneme.xlsx')
with pd.ExcelWriter(path, engine='xlsxwriter') as excel_writer:

# Write each DataFrame to a different sheet
    solution_df.to_excel(excel_writer, sheet_name='solution', index=False)
    result_df.to_excel(excel_writer, sheet_name='results', index=False)




new_format_array = np.hstack((new_format_array, np.zeros((new_format_array.shape[0], 1))))

# Iterate through each row in new_format_array
for row_idx in range(new_format_array.shape[0]):
    # Extract team1 and team2 from the current row
    team1 = int(new_format_array[row_idx, 1])
    team2 = int(new_format_array[row_idx, 2])

    # Find the corresponding row in results_df based on team1 and team2
    match_result = results_df[((results_df['Team 1'] == team1) & (results_df['Team 2'] == team2)) |
                               ((results_df['Team 1'] == team2) & (results_df['Team 2'] == team1))]['Result'].values

    # If a match result is found, update the new column in new_format_array
    if len(match_result) > 0:
        new_format_array[row_idx, -1] = match_result[0]